In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")


In [ ]:
X = df["question_text"]
y = df["target"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3))

In [ ]:
X_t = df_test["question_text"]
X_t.head()

In [ ]:
import time 
start_time = time.time()
XX_train = vectorizer.fit_transform(X)
XX_test = vectorizer.transform(X_t)
yy_train = y
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm, metrics
from sklearn.svm  import SVC


ros = RandomOverSampler(random_state=0)

XX_restrain, yy_restrain = ros.fit_resample(XX_train, yy_train)

mOver = MultinomialNB().fit(XX_restrain, yy_restrain)



In [ ]:
print("Train shape: ", XX_restrain.shape)
print("Test shape: ", XX_test.shape)

### Feature Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import time

s_time = time.time()
mod = LogisticRegression()
rfe = RFE(mod, 2000000)
fit = rfe.fit(XX_restrain, yy_restrain)
print("Num Features: %s" % (fit.n_featrues_))
print("--- %s seconds ---" % (time.time() - s_time))

In [ ]:
y_trainPred = mOver.predict(XX_restrain)
# TRAIN PERFORMANCE
print("Precision:" , metrics.precision_score(yy_restrain, y_trainPred, average='macro'))
print("Recall:" , metrics.recall_score(yy_restrain, y_trainPred, average='macro'))
print("Accuracy:", metrics.accuracy_score(yy_restrain, y_trainPred))
print("f-Score:", metrics.f1_score(yy_restrain, y_trainPred))

In [ ]:
y_testPred = mOver.predict(XX_test)

In [ ]:
print(mOver.class_log_prior_)

In [ ]:
mOverPriors = MultinomialNB(class_prior=[0.9, 0.1]).fit(XX_restrain, yy_restrain)

In [ ]:
y_trainPredPrior = mOverPriors.predict(XX_restrain)
# TRAIN PERFORMANCE
print("Precision:" , metrics.precision_score(yy_restrain, y_trainPredPrior, average='macro'))
print("Recall:" , metrics.recall_score(yy_restrain, y_trainPredPrior, average='macro'))
print("Accuracy:", metrics.accuracy_score(yy_restrain, y_trainPredPrior))
print("f-Score:", metrics.f1_score(yy_restrain, y_trainPredPrior))

In [ ]:
y_testPredPrior = mOverPriors.predict(XX_test)

In [ ]:
out_df = pd.DataFrame({"qid":df_test["qid"].values})
out_df['prediction'] = y_testPredPrior
out_df.to_csv("submission.csv", index=False)